# Reproducible investigations of maritime glaciers using open-source tools
This notebook ({nb-download}`download <IS2_OSS-FAIR-Resources_Workflow.ipynb>`) illustrates the use of multiple open-source tools (icepyx, IceFlow, SlideRule) for exploring an Alaskan maritime glacier.

The notebook was designed for a presentation at the June 2022 International Glaciological Society (IGS) [International Symposium on Maritime Glaciers](https://www.igsoc.org/event/international-symposium-on-maritime-glaciers) in Juneau, Alaska, USA.

**Symposium Abstract:**
Multiple open-source software (OSS) packages developed by and for the glaciological community enable rapid investigations of maritime glaciers. Focusing on Alaskan maritime glaciers, we illustrate how icepyx and other community-built software packages can be leveraged to quickly explore ICESat-2 data in combination with data from other sensors for a given glacier. The first tool showcased, the Python package icepyx, was created by the author in response to challenges faced by the glaciology community in accessing ICESat-2 data programmatically. With icepyx, we query and quickly visualize ICESat-2 data of the glacier. Then, we construct a time series of elevations spanning the ICESat, IceBridge, and ICESat-2 sensors using the NSIDC IceFlow package. Last, we customize our ICESat-2 data analyses with in-cloud processing using SlideRule. The workflow, encapsulated within an executable Jupyter Notebook, showcases the tools' ease of use for data access, analysis, and visualization while demonstrating the application of FAIR (Findable, Accessible, Interoperable, Reusable) principles and collaborative development in glaciological research.


### Tools Showcased
 1. [icepyx]()
 2. [IceFlow]()
 3. [SlideRule]()


### Objectives
 1. Showcase several open-source tools useful to glaciologists
 2. Demonstrate the application of FAIR principles in glaciological research
 3. Investigate a maritime glacier...

# TO DO:
- add links to tools
- update science and FAIR objectives
- add refs/source material
- figures/images (use table Mikala made)

## Maybe to-do:
- get RGI from NSIDC via CMR, if it's an option

### Environment

In [ ]:
# import needed packages
import geopandas as gpd
import icepyx as ipx


The Randolph Glacier Inventory ([RGI](https://nsidc.org/data/nsidc-0770)), part of Global Land Ice Measurements from Space ([GLIMS](https://www.glims.org/)) provides glacier outlines. Here we'll open the Alaska glacier outlines into a GeoPandas DataFrame.

In [ ]:
# get RGI glacier polygons
rgi_zip_fn = '01_rgi60_Alaska.zip'
url = 'https://www.glims.org/RGI/rgi60_files/' + rgi_zip_fn
ak_rgi_gdf = gpd.read_file(url)

In [ ]:
ak_rgi_gdf.head()

In [ ]:
ak_rgi_gdf.plot()

In [ ]:
# choose a glacier
# ak_rgi_gdf[~ak_rgi_gdf["Name"].isnull()]
glac = ak_rgi_gdf[ak_rgi_gdf["Name"] == "Mendenhall Glacier"]

In [ ]:
glac.plot()

### ICESat-2 data via icepyx

In [ ]:
# get exterior coordinates of the glacier polygon
poly = list(glac.geometry.values[0].exterior.coords)

In [ ]:
# create an icepyx Query object
is2_glac = ipx.Query(spatial_extent=poly, 
                     date_range=['2019-02-20','2019-02-28'], 
                     product="ATL06")

In [ ]:
# visualize our outline on a map
is2_glac.visualize_spatial_extent()

In [ ]:
# quick-view available ICESat-2 data with icepyx


In [ ]:
# download a longer time-series with IceFlow

In [ ]:
# get higher-resolution ICESat-2 data with SlideRule

**Credits**
* notebook by: Jessica Scheick
* notebook contributors: 
* source material: []() by ???